# TAURUS + NSF REU 2022
# Introduction to Python Day 3
# Statistics, Interpolation & Fitting Data

Welcome back again! So far we have covered basic syntax, arrays, if statements, for loops, functions, and plotting. Now we can start building some data analysis skills using statistics. 

Python has lots of built in functions for generating random data. Today let's start with generating random values from a Gaussian distribution. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(42)

In [ ]:
gaussian = np.random.normal(0, 2, 1000) 
#Gaussian centered on 0 with a std. dev. of 2, sampled 1000 times

## Histograms 

A plot you'll probably make quite often is a histogram. For this, use plt.hist().

    plt.hist(data, bins, stacked=True/False, density=True/False)
    
The "bins" argument can either be an array, or simply setting "bins=x" for a number of evenly spaced bins. Setting stacked and density to True will normalize your data to sum to 1.

In [ ]:
plt.hist(gaussian, bins=20, stacked=True, density=True)
plt.grid(True)
plt.show()

Just like with normal plotting, there are lots of customizable options for histograms, including:

    facecolor: string argument giving color of histogram
    edgecolor: string argument giving edge color
    alpha: opacity, ranging from 0 to 1
    hatch: a histogram filling style, such as '\\', '//', '.', '..', etc
    histtype: bar, step
    density: True/False
    stacked: True/False
    cumulative: True/False
    
Full details here: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html
    
## Gaussian Fitting

Let's try fitting the Gaussian. This is a little bit redundant since we already know it's a Gaussian distribution, but this example should illustrate how the fitting process works. Let's use a package from scipy:

In [ ]:
from scipy.stats import norm

Pro-tip: if you're ever unsure about how to use a function, type ? after the function name to call up the docstrings.

To fit a Gaussian, simply use norm.fit(data). This will return the mean and standard deviation of the fit. np.average() and np.std() will give the mean and standard deviation of the actual data.

In [ ]:
mu, sigma = norm.fit(gaussian)
print(mu, sigma)

This did pretty well. It performs better the larger the sample you give it. 

Next, to overplot the Gaussian fit, you can calculate the probability density function (PDF).

In [ ]:
x = np.linspace(-5, 5, 1000)
pdf = norm.pdf(x, mu, sigma)

plt.plot(x, pdf, 'k-', linewidth=2)
plt.hist(gaussian, bins=50, color='mediumpurple', alpha=0.4, density=True, fill=True, histtype='step')
plt.title("Fit results: $\mu$ = %.2f, $\sigma$ = %.2f" % (mu, sigma)) 
#Pro-tip: this prints out the values of mu and sigma to 2 decimal places.

plt.show()

## Interpolating Data

You may encounter a situation in your research where you will need to interpolate a model to interpret your data. The scipy library has many different interpolating (and extrapolating) algorithms, but the simplest is a 1D interpolation, called interp1d. 

Let's first generate a well-behaved function that will be easy to interpolate along.

In [ ]:
xarray = np.linspace(-4*np.pi, 4*np.pi, 30)
yarray = np.sin(xarray)

plt.plot(xarray, yarray, 'o')
plt.show()

What do you notice about this plot? Well, at first glance it doesn't really look like a sine curve because it is poorly sampled. Of course, since we already know it's a sine curve, we could just sample the function better, but say you didn't have such a nice function. Interp1d comes to the rescue! 

In [ ]:
from scipy.interpolate import interp1d

There are a few steps to interpolation. First, the interp1d function interpolates your original y array. Then, you need to define the new array along which to interpolate, which must completely fit inside the original array (or else it's extrapolation). We'll take the same array but grid it more finely. Finally, you create a new y array using the interpolated function.

In [ ]:
interp = interp1d(xarray, yarray)
xnew = np.linspace(-4*np.pi, 4*np.pi, 1000)
ynew = interp(xnew)

plt.plot(xnew, ynew, '--')
plt.plot(xarray, yarray, 'or')
plt.show()

Now it looks much closer to a sine curve!

## Loading data files into Python 

Up until now, none of the exercises have required you to load in an outside file to the notebook, but this is definitely something you'll need to learn! You can use np.genfromtxt, np.loadtxt, or a library called pandas if you're feeling advanced. 

If your file is just a 1D array of floats, then simply defining a variable with np.genfromtxt('filename') will suffice. Other arguments include "dtype" which allows you to specify that they're strings or integers, and in a moment I'll show you how to transpose a text file.

First let's load up the spectrum we are going to work with. Python reads by rows, not columns, but if you want separate arrays indicating the columns such as wavelengths, flux, and flux errors, you can transpose it, by doing np.genfromtxt('filename').T. You can define it all in one line, like so:

x, y, z = np.genfromtxt('file_with_three_columns.txt').T

###  Load up the file "test_spectrum.txt", whose three columns are wavelength, flux, and error.

In [ ]:
# load file here

# Fitting a Spectrum

Finally we can put what we've learned together to do a real life astronomy example! We will do this exercise together.

Here is a sample spectrum which features a prominent spectral emission line. Our job will be to fit the line with a Gaussian model to compute its peak flux and FWHM. 


Let's plot up what we have so far. This is a great opportunity to show you another useful plotting tool: errorbars! 

Instead of doing plt.plot, you can do plt.errorbar, which plots both the data and the errorbars. The first two arguments are x and y like in plt.plot, and the next two arguments are xerr and yerr. You can specify the color of the errorbars using the keyword argument (kwarg) "ecolor", and you can put hats on the errorbar with "capsize."

### Plot the spectrum with y errors (it doesn't have x errors) with the data in blue and the errorbars in magenta.

In [ ]:
# your plot here


We're going to use the optimize function from scipy. This allows you to fit pretty much any kind of function you want, but here we know that the emission line has a Gaussian profile. 

Many spectral lines are well-described by a Gaussian curve, and lines that deviate from a Gaussian are often astronomically interesting. So the first step in many spectroscopic projects is to see if your line is Gaussian.

$N(x, \mu, \sigma) = \frac{1}{\sqrt{2 \pi}\sigma}e^{-\frac{(x-\mu)^2}{2\sigma^2}}$

This means that $\mu$, the mean of the Gaussian, is the wavelength at which the line peaks. This helps you decide what line it is (Ly$\alpha$, carbon monoxide, nitrogen, etc.)

$\sigma$ is the width of the line -- spectral lines have natural shifts a bit to the left and a bit to the right of the "true" wavelength it should peak at. There are astronomical reasons why your line would be extremely wide, or skewed to one side.

f0 is the peak height of the line, or the flux at which your line peaks -- how bright your astronomical object is. The reason we call it "f0" instead of the full Gaussian term is we don't want it to get caught up in the value of $\sigma$.

$N(x, f_0, \mu, \sigma) = f_0 e^{-\frac{(x-\mu)^2}{2\sigma^2}}$

### Import the optimize function from scipy as "opt." Using what you learned yesterday, define a Gaussian function which takes as its arguments x, $f_0$, $\mu$, and $\sigma$. (Here, $f_0$ is a normalizing factor in front of the Normal distribution, used in place of the $(\sigma * \sqrt{2 \pi})^{-1}$ term.) When you use optimize, it's important that the first argument is the array you're fitting along, so here that is x. 

In [ ]:
# import your function here

# define your gaussian here (hint: if you did your homework you did this already!)

The optimize function finds the best fit parameters as well as the covariance matrix for those parameters. p0 is the initial guess for the fit - this is important because a bad or no first guess will cause a bad fit. So the rest of the fit goes like so:

See if you can guess a good p0!

*Make sure the order of p0 is the same as the order of the arguments in the function!*

In [ ]:
bfpars, covar = opt.curve_fit(gauss, wave, flux, p0=[10700, 100, 2.0e-18], sigma=error) #edit p0

print(bfpars)
#so now we see the line peaks at 10,704 Angstrom, it has a width of 219 Angstrom, 
#and it has a peak flux of 1.88e-18 erg s^-1 cm^-2 AA^-1

"Bfpars" is now an array containing the best fit parameters, in the same order as your function. So in my case, bfpars[0] is $\mu$, bfpars[1] is $\sigma$, and bfpars[2] is f0.

Try printing out the best fit parameters. Do the values make sense to you? 

Let's define a higher resolution wavelength grid we can use to overlay the fit. We can also zoom in on the line since the rest of the spectrum is noise.

Below, x should be a grid that is centered on $\mu$, and is plotted from -10$\sigma$ to 10$\sigma$ (so make sure your index values are correct, depending on how you defined the Gaussian function -- mine has $\mu$, $\sigma$, f0 in that order).  This just means I want you to evaluate the Gaussian right where the emission line occurs.

In [ ]:
x = np.linspace(bfpars[0]-10*bfpars[1], bfpars[0]+10*bfpars[1], 1000) 
#Center the grid on the wavelength of the line
#and go +- 10*sigma from there

modelfit = gauss(x, bfpars[0], bfpars[1], bfpars[2]) 
#This is the Gaussian curve with the best fit parameters

## Saving Data Files

The syntax for writing files is similar to loading them, with many customizable options in np.savetxt(filename, data). The arguments are as follows:

    fname: the path and filename you want to save to
    X: the array(s) being saved
    fmt: the format of each column of data, e.g. "%s" for strings or "%.2f" for a floating point with 2 decimal places
    delimiter: can leave this blank for evenly spaced columns, but you could use ',' for comma-separated values
    header: a string that might contain info about the file or the names of columns, like '# ra dec velocity'
    
    
Save the modelfit parameters to a text file in this directory.
    

In [ ]:
# save file here

### Your final task, plot the model over the data by including both plots in the same cell, and make sure you zoom in on the line by choosing a wise range for xlim. 

Spectra are typically viewed along a long x-axis: try changing the figure size.

Congrats, you've done your first bit of science with Python!

In [ ]:
# solution here

# Exercises

## Exercise 1: Model Fitting

There is a file in this directory called "histogram_exercise.dat" which consists of of randomly generated samples from a Gaussian distribution with an unknown $\mu$ and $\sigma$. Using what you've learned about fitting data, load up this file using np.genfromtxt, fit a Gaussian curve to the data and plot both the curve and the histogram of the data. As always, label everything, play with the colors, and choose a judicious bin size.

Hint: if you attempt to call a function from a library or package that hasn't been imported, you will get an error.

## Exercise 2: Interpolation

Create a 1D interpolation along these arrays.

a. Plot both the data (as points) and the interpolation (as a dotted line).

b. Also plot the value of the interpolated function at x=325. What does the function look like to you?

In [ ]:
x = np.array([0., 50., 100., 150., 200., 250., 300., 350., 400., 450., 500])
y = np.array([0., 7.071, 10., 12.247, 14.142, 15.811, 17.321, 18.708, 20., 21.213, 22.361])

#Solution Here


## Excercise 3: MCMC

A big part of astronomy is moving towards implementing Bayesian techniques. These techniques are useful as you can use prior information to inform new predictions, can assess the likelihod of the data you collected and provides a natural way to quantify the uncertainty on derived parameters. For this problem we will code up our very own MCMC algorithm using a technique called Metropolis Hastings algorithm.

We will walk throught he steps of how it works and you are tasked with translating it into code.

The Metropolis Hasting Sampling algorithm is an algorithm that takes your current start position and checks it against a trial value which we specify by inorporating a jump size to jump from our current value to another one. We then check to see if the new trial is more likely than the other one and if it is we keep the value but if it is not we reject the trial and stay at the current value and try again. We do this until we store our 10,000 values. 

For this problem we want to use MCMC to fit the Mass-Radius relation for M-dwarf Stars and see what values for the slope and y-intercept we get. The Mass-Radius relation is given by $R(M) = aM + b$ and our job is to find the best fit a and b values for the given data. In the directory is a file titled MCMC_Data.txt you will use this as this has the R, M and $\sigma_R$ values which you will need in the MCMC below. 


Steps: 

1. We need to decide how many values we want to ultimately aquire as part of the MCMC, it is good practice to start small so say L = 100 and then once the code works increase it to L = 100,000
2. We will need to jump from our current value to a test value so for this we need to specify a jump size which will be done by randomly generating a test value from a gaussian centered on your current value with a standard devation which we will provide use [$\sigma_a = 0.002, \sigma_b = 0.001$]
3. Come up with an initial guess for the parameters, i recommend starting the slope at a value near 1.04 and the y-intercept at 0.001. But feel free to play around with these starting values. But do note the step size of your $\sigma$ values as this assumes that you were close to the correct value. 
4. Start a loop and specify it to stop after you have accepted the number L from Part 1. It should start off as a [1 X 2] matrix but you will eventually generate an [L X 2] Matrix.
5. Now we need to vary one parameter and check if we accept it or reject it. At this stage generate a random number between 0-1 and round it. This will be the parameter to vary as a 0 means you will vary the first value and 1 means you will vary the second value
6. Now that we have which parameter we will vary we need to draw a trial value, to select a trial value we take the current value and draw from a gaussian with the corresponding $\sigma$ from step 2. So if the first value is the one that gets varied you have a current trial [$a_c, b_c$] and your trial test is [$a_t, b_c$] with $a_t$ being a random number drawn from the Gaussian $a_t = N(\mu = a_c, \sigma = \sigma_a)$, if the second value is varied then you sample from a Gaussian $b_t = N(\mu = b_c, \sigma = \sigma_b)$ and have [$a_c, b_t$].
7. The goal is to compare which value has the highest likelihood between the current or trial values(ie: comparing [$a_c, b_c$] to [$a_t, b_c$] or [$a_c, b_c$] to [$a_c, b_t$]). At this step generate two sets of model predictions for the given $x_i$. One with the current parameter values $m_{c, i} = a_c x_i + b_c$ and another with the trial parameter values $m_{t, i} = a_t x_i + b_c$ if the first parameter is changing, $m_{t, i} = a_c x_i + b_t$ if the second parameter is changing)
8. Compute the $\chi^2$ for the current and trial runs using $\chi_c^2 = \sum_i^N \frac{(R_i - m_{c, i})^2}{\sigma_{R, i}^2}$ and trial run being $\chi_t^2 = \sum_i^N \frac{(R_i - m_{t, i})^2}{\sigma_{R, i}^2}$
9. Compute the ratio r = $e^{-\chi_t^2/2}/e^{-\chi_c^2/2} = e^{-(\chi_t^2 - \chi_c^2)/2}$
10. Draw a random number P from a uniform distribution between 0 and 1, then compare if P < r (from part 9) and if it is accept the trial values and add them to your matrix, if P > r we reject the trial values and start from step 5 again
11. Repeat steps 5-10 until you reach the desired L from part 1

In [ ]:
#Read in the data from the MCMC_Chain.txt file


In [ ]:
def Model():
    
    '''
    This will be the function that we will want to fit. In this particular example 
    we want to fit the Mass-Radius Relation R(M) = aM + b
    
    Inputs
    ----------------
    
    
    Output(s)
    ----------------
    
    '''
    
    return 


def Chi2():
    
    '''
    Function that will compute the chi2 for a given R, R_err, and model
    R and R_err are given in the MCMC_Chain.txt file
    
    Inputs
    -----------
    
    
    Output(s)
    -----------
    
    
    '''
    
    return 

In [ ]:
# define a dummy array that will store the values of the MCMC Chain
# This should be an L x 2 array/matrix
# eventually the L should be 100000 but start of with 100 to make sure the code is working
# then work up to L of 100000
MCMC_chains = np.zeros()

# Sigma Declarations defined above
sigmas = [ , ]

##############
#Write the Metropolis-Hasting Sampling Algorithm here
##############

# Initial Starting Values
values = [1.04, 0.001]

#This counter will be the index for the rows in the MCMC chains that get accepted
#be sure to update this as this will keep track of how many chains get accepted 
counter = 0

#Decide which loop you want to use for this (ie: For-Loop/While-Loop) Think about the 
#acceptance will we always accept?

    # In the Loop
    # Generate a random number from 0-1 and round it, if 0 we will alter the a-parameter
    # if 1 we will alter the b-parameter

    #Now that you have which parameter we will vary
    #generate a new trial parameter by sampling from a gaussian 
    #centered on the current value with the appropriate sigma for the parameter
    
    #generate two model predictions
    #1. using the current parameter values
    #2. using the trial parameters
    
    #Compute the chi2 between the two models
    
    #Compute the ratio r defined above
    
    # Randomly sample from a uniform distribution between 0-1 and compare 
    # the random sample to the ratio r
    # if the random number is less than r 
    # We accept the trial parameters and store them in the MCMC_chains variable
    # if the random number is larger than r we reject and keep the current parameters
    #Then we try again note that we do not store here
    
    #repeat these steps in the loop until your MCMC chain has the desired L entries
    
    

In [ ]:
#During the MCMC fitting we need to get rid of the first few 1000 entries
#the reason for this is that the chain had not yet converged so we remove these
#with an L of 100000 remove the first 10000 values

param1_burnin = #this would be column 1 in MCMC_chain
param2_burnin = #this would be column 2 in MCMC_chain

In [ ]:
#To check if our values have converged lets plot them
plt.figure(figsize = (12, 7))
plt.plot(param1_burnin)
plt.show()

In [ ]:
#To check if our values have converged lets plot them
plt.figure(figsize = (12, 7))
plt.plot(param2_burnin)
plt.show()

In [ ]:
#If everything ran correctly these should look like shot noise 
#we can then make these into distributions and quantify the median as our value 
#and the error as the 16th and 84th percentile interval

In [ ]:
#Make a histogram of each quantity and make sure to make the plot as informative as possible
#ie: include title, x and y labels and any legends if necessary

